# Spatial

Here, you'll learn how to manage a growing number of spatial datasets as a single queryable collection:

1. curate and ingest spatial data (![spatial1/4](https://img.shields.io/badge/spatial1/4-lightgrey))
2. query & analyze spatial datasets ([![spatial2/4](https://img.shields.io/badge/spatial2/4-lightgrey)](/spatial2))
3. load the collection into memory & train a ML model ([![spatial3/4](https://img.shields.io/badge/spatial3/4-lightgrey)](/spatial3))
4. create and share interactive visualizations with vitessce ([![spatial4/4](https://img.shields.io/badge/spatial4/4-lightgrey)](/spatial4))


```{toctree}
:maxdepth: 1
:hidden:

spatial2
spatial3
spatial4
```

Spatial omics data integrates molecular profiling (e.g., transcriptomics, proteomics) with spatial information, preserving the spatial organization of cells and tissues.
It enables high-resolution mapping of molecular activity within biological contexts, crucial for understanding cellular interactions and microenvironments.

Many different spatial technologies such as ... exist, but here, we focus on a 10X Xenium dataset 

In [ ]:
# !pip install 'lamindb[jupyter,bionty]' spatialdata[spatialdata-io,spatialdata-plot]
!lamin init --storage ./test-spatial --schema bionty

In [ ]:
import lamindb as ln
import bionty as bt
import pandas as pd
from spatialdata.datasets import blobs
import spatialdata_plot
import warnings

warnings.filterwarnings("ignore")

ln.track()

## An example spatial dataset

In [ ]:
# Replace this with a proper dataset from for example https://spatialdata.scverse.org/en/latest/tutorials/notebooks/datasets/README.html

sdata = blobs()
sdata.attrs["sample"] = {
    "assay": "Visium Spatial Gene Expression",
    "disease": "Alzheimer's dementia",
    "developmental_stage": "very early",
}
sdata.tables["table"].var.index = [
    "ENSG00000139618",  # BRCA2
    "ENSG00000157764",  # BRAF
    "ENSG00000999999",  # Does not exist - to test add new
]
sdata.tables["table"].obs["sample_region"] = pd.Categorical(
    ["region 1"] * 13 + ["region 2"] * 13
)

In [ ]:
curator = ln.Curator.from_spatialdata(
    sdata,
    var_index={"table": bt.Gene.ensembl_gene_id},
    categoricals={
        "sample": {
            "assay": bt.ExperimentalFactor.name,
            "disease": bt.Disease.name,
            "developmental_stage": bt.DevelopmentalStage.name,
        },
        "table": {"sample_region": ln.ULabel.name},
    },
    organism="human",
)

curator.validate()
curator.add_new_from_var_index("table")
curator.add_new_from(key="developmental_stage", accessor="sample")
curator.add_new_from(key="sample_region", accessor="table")

curator.standardize(key="disease", accessor="sample")

artifact = curator.save_artifact(description="blob spatialdata")
artifact.describe()

In [ ]:
sdata.pl.render_images("blobs_multiscale_image").pl.show()

In [ ]:
# clean up test instance
!rm -rf test-spatial
!lamin delete --force test-spatial